In [4]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [1]:
# data loading / manip
import mne
import re
import numpy as np
import os
import csv
import h5py
import warnings
from tqdm.notebook import tqdm
import random
import itertools as itools
import pandas as pd
import time
from img_pipe import img_pipe
# plotting
from matplotlib import pyplot as plt
from matplotlib import rcParams, cm
plt.style.use('seaborn')
rcParams['pdf.fonttype'] = 42
import seaborn as sns
import sys
sys.path.append(os.path.join(git_path,"analysis","mtrf"))
import mtrf_utils

## Different models, different hypotheses
* Model 1: with phnfeat + spk + mic + el + sh.
    * Hypothesis: we see the same tuning but a reduction in amplitude.
    * nfeats = 18 (14 phnfeat + 4 task)
* Model 2: phnfeat_spk + phnfeat_mic + phnfeat_any + mic + el + sh.
    * Hypothesis: the tuning is changing between conditions.
    * nfeats = 46 (14 phnfeat_all + 14 phnfeat_mic + 14 phnfeat_spkr + 4 task)
* Model 3: phnfeat + spk + mic.
    * Hypothesis: Predictable/unpredictable don't make a difference.
    * nfeats = 16 (model 1- 2 feat (el/sh))
* Model 4: phnfeat + el + sh.
    * Hypothesis: Perc/prod don't make a difference.
    * nfeats = 16 (model 1- 2 feat (spkr/mic))

In [5]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

### Step 1. Load stim/resp t/v/ split from hdf5
**If you haven't made these, run `make_stim_resp.ipynb` first!**

Here we load in the stim/resp training/validation split from an hdf5 file, then index those files along specific stimulus features given the particular model we're interested in running.

In [6]:
tStims, tResps, vStims, vResps = dict(), dict(), dict(), dict()
pbar = tqdm(subjs)
for s in pbar:
    tStims[s], tResps[s], vStims[s], vResps[s] = dict(), dict(), dict(), dict()
    for m in models:
        pbar.set_description(f"Loading model inputs for {s} {m}")
        # Update this file location accordingly on your local machine!
        model_input_h5_fpath = os.path.join(git_path,"analysis","mtrf","h5","model_inputs",
                                            f"{s}_model_inputs.hdf5")
        tStims[s][m], tResps[s][m], vStims[s][m], vResps[s][m] = mtrf_utils.load_model_inputs(
            model_input_h5_fpath, m, mode='ecog')
        print(s, m, "t/v stim:", tStims[s][m].shape, vStims[s][m].shape, "||",
              "t/v resp:", tResps[s][m].shape, vResps[s][m].shape)

  0%|          | 0/16 [00:00<?, ?it/s]

S0004 model1 t/v stim: (82706, 18) (22146, 18) || t/v resp: (82706, 81) (22146, 81)
S0004 model2 t/v stim: (82706, 46) (22146, 46) || t/v resp: (82706, 81) (22146, 81)
S0004 model3 t/v stim: (82706, 16) (22146, 16) || t/v resp: (82706, 81) (22146, 81)
S0004 model4 t/v stim: (82706, 16) (22146, 16) || t/v resp: (82706, 81) (22146, 81)
S0006 model1 t/v stim: (97499, 18) (27205, 18) || t/v resp: (97499, 104) (27205, 104)
S0006 model2 t/v stim: (97499, 46) (27205, 46) || t/v resp: (97499, 104) (27205, 104)
S0006 model3 t/v stim: (97499, 16) (27205, 16) || t/v resp: (97499, 104) (27205, 104)
S0006 model4 t/v stim: (97499, 16) (27205, 16) || t/v resp: (97499, 104) (27205, 104)
S0007 model1 t/v stim: (113073, 18) (33055, 18) || t/v resp: (113073, 96) (33055, 96)
S0007 model2 t/v stim: (113073, 46) (33055, 46) || t/v resp: (113073, 96) (33055, 96)
S0007 model3 t/v stim: (113073, 16) (33055, 16) || t/v resp: (113073, 96) (33055, 96)
S0007 model4 t/v stim: (113073, 16) (33055, 16) || t/v resp: (

### Step 2. Run the mTRF

In [7]:
# mTRF parameters
nboots = 10
delay_min, delay_max = -0.3, 0.5
delays = np.arange(np.floor(delay_min*100),np.ceil(delay_max*100),dtype=int)
ndelays = len(delays)
# Change the alphas if the output looks choppy (it's a smoothing parameter basically)
# We want our subj_best_alpha to be in the middle of our alpha range, not at one end
alpha_min, alpha_max = -4, 4
alphas = np.hstack((0, np.logspace(alpha_min,alpha_max,20)))

In [8]:
# Run the mTRF
corrs, wts, best_alphas = dict(),dict(),dict()
pbar = tqdm(subjs)
for s in pbar:
    corrs[s], wts[s], best_alphas[s] = dict(),dict(),dict()
    for m in models:
        pbar.set_description(f"Fitting mTRF for {s} {m}")
        subj_corrs, subj_wts, _, _, _, _, subj_best_alphas = mtrf_utils.mtrf( # don't save stim/resp
            tResps[s][m], tStims[s][m],
            vResp = vResps[s][m], vStim = vStims[s][m],
            nboots=nboots, delay_min=delay_min, delay_max=delay_max, alphas=alphas,
            flip_resp = True, sfreq = 100
        )
        print(
            f"{s} {m}: Best alpha: {subj_best_alphas[0]} (should be in between {alphas[0]} and {alphas[-1]})"
        )
        # Write model output to dicts
        corrs[s][m] = subj_corrs[0] # R-value between predicted and actual EEG at each channel
        wts[s][m] = subj_wts[0].reshape((
            # delays x feats x chans
            ndelays,tStims[s][m].shape[1],tResps[s][m].shape[1]
        )) 
        best_alphas[s][m] = subj_best_alphas # Regularization parameter that yielded best model fit

  0%|          | 0/16 [00:00<?, ?it/s]

81 82706
Using training/validation split passed into the func...
(82706, 1440) (22146, 1440)
(82706, 81) (22146, 81)


INFO:counter:1/10 items complete (4.25 seconds/item, 00:00:38 remaining)
INFO:counter:2/10 items complete (4.27 seconds/item, 00:00:34 remaining)
INFO:counter:3/10 items complete (4.23 seconds/item, 00:00:29 remaining)
INFO:counter:4/10 items complete (4.20 seconds/item, 00:00:25 remaining)
INFO:counter:5/10 items complete (4.20 seconds/item, 00:00:21 remaining)
INFO:counter:6/10 items complete (4.21 seconds/item, 00:00:16 remaining)
INFO:counter:7/10 items complete (4.22 seconds/item, 00:00:12 remaining)
INFO:counter:8/10 items complete (4.22 seconds/item, 00:00:08 remaining)
INFO:counter:9/10 items complete (4.22 seconds/item, 00:00:04 remaining)
INFO:counter:10/10 items complete (4.23 seconds/item, 00:00:00 remaining)


Cmode = False
Number of time points is greater than the number of features
stim shape (not cmode): 
(82706, 1440)
Covmat shape: 
(1440, 1440)
(1440, 1440) (1440, 82706) (82706, 81)
wt shape:
(1440, 81)
(1440, 81)
S0004 model1: Best alpha: 10000.0 (should be in between 0.0 and 10000.0)
81 82706
Using training/validation split passed into the func...
(82706, 3680) (22146, 3680)
(82706, 81) (22146, 81)


INFO:counter:1/10 items complete (16.65 seconds/item, 00:02:29 remaining)
INFO:counter:2/10 items complete (15.96 seconds/item, 00:02:07 remaining)
INFO:counter:3/10 items complete (15.65 seconds/item, 00:01:49 remaining)
INFO:counter:4/10 items complete (15.47 seconds/item, 00:01:32 remaining)
INFO:counter:5/10 items complete (15.32 seconds/item, 00:01:16 remaining)
INFO:counter:6/10 items complete (15.24 seconds/item, 00:01:00 remaining)
INFO:counter:7/10 items complete (15.16 seconds/item, 00:00:45 remaining)
INFO:counter:8/10 items complete (15.10 seconds/item, 00:00:30 remaining)
INFO:counter:9/10 items complete (15.02 seconds/item, 00:00:15 remaining)
INFO:counter:10/10 items complete (15.04 seconds/item, 00:00:00 remaining)


...

Cmode = False
Number of time points is greater than the number of features
stim shape (not cmode): 
(32935, 3680)
Covmat shape: 
(3680, 3680)
(3680, 3680) (3680, 32935) (32935, 255)
wt shape:
(3680, 255)
(3680, 255)
TCH14 model2: Best alpha: 10000.0 (should be in between 0.0 and 10000.0)
255 32935
Using training/validation split passed into the func...
(32935, 1280) (9279, 1280)
(32935, 255) (9279, 255)


INFO:counter:1/10 items complete (1.79 seconds/item, 00:00:16 remaining)
INFO:counter:2/10 items complete (1.76 seconds/item, 00:00:14 remaining)
INFO:counter:3/10 items complete (1.77 seconds/item, 00:00:12 remaining)
INFO:counter:4/10 items complete (1.77 seconds/item, 00:00:10 remaining)
INFO:counter:5/10 items complete (1.77 seconds/item, 00:00:08 remaining)
INFO:counter:6/10 items complete (1.77 seconds/item, 00:00:07 remaining)
INFO:counter:7/10 items complete (1.76 seconds/item, 00:00:05 remaining)
INFO:counter:8/10 items complete (1.76 seconds/item, 00:00:03 remaining)
INFO:counter:9/10 items complete (1.76 seconds/item, 00:00:01 remaining)
INFO:counter:10/10 items complete (1.76 seconds/item, 00:00:00 remaining)


Cmode = False
Number of time points is greater than the number of features
stim shape (not cmode): 
(32935, 1280)
Covmat shape: 
(1280, 1280)
(1280, 1280) (1280, 32935) (32935, 255)
wt shape:
(1280, 255)
(1280, 255)
TCH14 model3: Best alpha: 10000.0 (should be in between 0.0 and 10000.0)
255 32935
Using training/validation split passed into the func...
(32935, 1280) (9279, 1280)
(32935, 255) (9279, 255)


INFO:counter:1/10 items complete (1.75 seconds/item, 00:00:15 remaining)
INFO:counter:2/10 items complete (1.75 seconds/item, 00:00:13 remaining)
INFO:counter:3/10 items complete (1.75 seconds/item, 00:00:12 remaining)
INFO:counter:4/10 items complete (1.76 seconds/item, 00:00:10 remaining)
INFO:counter:5/10 items complete (1.75 seconds/item, 00:00:08 remaining)
INFO:counter:6/10 items complete (1.75 seconds/item, 00:00:07 remaining)
INFO:counter:7/10 items complete (1.75 seconds/item, 00:00:05 remaining)
INFO:counter:8/10 items complete (1.75 seconds/item, 00:00:03 remaining)
INFO:counter:9/10 items complete (1.75 seconds/item, 00:00:01 remaining)
INFO:counter:10/10 items complete (1.75 seconds/item, 00:00:00 remaining)


Cmode = False
Number of time points is greater than the number of features
stim shape (not cmode): 
(32935, 1280)
Covmat shape: 
(1280, 1280)
(1280, 1280) (1280, 32935) (32935, 255)
wt shape:
(1280, 255)
(1280, 255)
TCH14 model4: Best alpha: 0.0 (should be in between 0.0 and 10000.0)


In [9]:
for s in subjs:
    for m in models:
         print(
            f"{s} {m} Alpha min: {alphas[0]} || Alpha max: {alphas[-1]} || Best alpha (should be in min-max range): {best_alphas[s][m][0]}"
         )
    print("")

S0004 model1 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 10000.0
S0004 model2 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 10000.0
S0004 model3 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 10000.0
S0004 model4 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 10000.0

S0006 model1 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 1438.44988828766
S0006 model2 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 1438.44988828766
S0006 model3 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 1438.44988828766
S0006 model4 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 545.5594781168514

S0007 model1 Alpha min: 0.0 || Alpha max: 10000.0 || Best alpha (should be in min-max range): 3792.690190732246
S0007 model2 Alpha min: 0.0 || Al

### Step 2a. Re-fit STRFs for subj/models with poor alpha range
I.e., the best alpha is either the min or the max value.

### 3. Save model results

In [22]:
# Save weights to hdf5 file
force_overwrite = True
pbar = tqdm(subjs)
for s in pbar:
    model_output_h5_fpath = os.path.join(git_path,"analysis","mtrf","h5","weights",f"{s}_weights.hdf5")
    for m in models:
        pbar.set_description(f"Saving model results for {s} {m}")
        # Update this file location accordingly on your local machine!
        if os.path.isfile(model_output_h5_fpath):
            with h5py.File(model_output_h5_fpath,'a') as f:
                if m not in f.keys():
                    f.create_dataset(m, data = wts[s][m])
                elif force_overwrite:
                    f[m][:] = wts[s][m]
                else:
                    print(
                        f"{s} {m} weights already saved to hdf5. To force an overwrite, set 'force_overwrite' to True."
                    )
        else:
            with h5py.File(model_output_h5_fpath,'w') as f:
                f.create_dataset(m, data = wts[s][m])    

  0%|          | 0/16 [00:00<?, ?it/s]

In [23]:
force_overwrite = True
results_csv_fpath = os.path.join(git_path,"analysis","mtrf","results.csv")
df = pd.read_csv(results_csv_fpath)
pbar = tqdm(subjs)
for s in pbar:
    for m in models:
        pbar.set_description(f"Saving alphas/corrs for {s} {m} to csv")
        for i,ch in enumerate(ch_names[s]):
            if len(df[(df['subject']==s) & (df['model']==m) & (df['channel']==ch)]) == 0:
                # Data does not exist in dataframe so let's add it
                # using nan as pval as a placeholder as we still need to bootstrap
                new_row = pd.DataFrame({
                    'subject':[s], 'model':[m], 'channel':[ch],
                    'r_value':[corrs[s][m][i]], 'p_value':["nan"], "best_alpha":[best_alphas[s][m][0]]
                })
                df = df.append(new_row,ignore_index=True)
            elif force_overwrite:
                tgt_row = df[(df['subject']==s) & (df['model']==m) & (df['channel']==ch)]
                df.loc[tgt_row.index, 'r_value'] = corrs[s][m][i]
                df.loc[tgt_row.index, 'best_alpha'] = best_alphas[s][m][0]
                df.loc[tgt_row.index, 'p_value'] = "nan"
            else:
                print(
                    f"data for {s} {m} {ch} already exists. To overwrite, set 'force_overwrite' to True"
                )
df.to_csv(results_csv_fpath,index=False)

  0%|          | 0/16 [00:00<?, ?it/s]